<a href="https://colab.research.google.com/github/jumafernandez/clasificacion_correos/blob/main/notebooks/jaiio/03-modelos/08-ensamble%2BBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sistema de votación (LR+SS3+TFIDF) + BERT

En esta notebook se presetan los experimentos a partir de los datos etiquetados automáticamente a partir de las _features_ extraidas del train dataset con todas las estrategias + la construcción de un clasificador con BERT.

## 0. Configuración de la ejecución

A continuación se genera la configuración de las estrategias que integrarán el sistema de votación para la obtención de instancias:

In [37]:
# Posibilidades : lr, ss3, tfidf
# Combinaciones:
# lr-ss3
# lr-tfidf
# ss3-tfidf
# lr-ss3-tfidf

estrategias_feature_extraction = ['lr', 'tfidf']
ETIQUETADO_MANUAL = True

# Para el sistema de votación queda fijo
CANTIDAD_INSTANCIAS = 200

Se definen algunas características fijas:

In [38]:
# El archivo de test y el train con etiquetado manual es siempre el mismo
TRAIN_FILE_MANUAL = 'correos-train-jaiio-80.csv'
TEST_FILE = 'correos-test-jaiio-20.csv'
atributos_df = ['consulta', 'dia_semana', 'semana_del_mes', 'mes', 'cuatrimestre',
                  'anio', 'hora_discretizada', 'dni_discretizado', 'legajo_discretizado',
                  'posee_legajo', 'posee_telefono', 'carrera_valor', 'proveedor_correo',
                  'cantidad_caracteres', 'proporcion_mayusculas', 'proporcion_letras',
                  'cantidad_tildes', 'cantidad_palabras', 'cantidad_palabras_cortas',
                  'proporcion_palabras_distintas', 'frecuencia_signos_puntuacion',
                  'cantidad_oraciones', 'utiliza_codigo_asignatura', 'score']

Y se generan los datos en función de la cantidad de estrategias a utilizar:

In [39]:
# Se genera el nombre del archivo de instancias a ejecutar en función de las estrategias

# En caso que se realice el ensamble entre las n estrategias
if len(estrategias_feature_extraction)>=1:
  TRAIN_FILE_E0 = f'dataset-{estrategias_feature_extraction[0]}-200-prep.csv'
  texto = f'Los dataset a utilizar son: \n\t{TRAIN_FILE_E0}'
  # Se define el if para que ante reiteradas ejecuciones de la notebook no se vuelva a insertar
  if 'clase_e0' not in atributos_df:
    atributos_df.append('clase_e0')

if len(estrategias_feature_extraction)>=2:
  TRAIN_FILE_E1 = f'dataset-{estrategias_feature_extraction[1]}-200-prep.csv'
  texto = texto + f'\n\t{TRAIN_FILE_E1}'
  # Se define el if para que ante reiteradas ejecuciones de la notebook no se vuelva a insertar
  if 'clase_e1' not in atributos_df:
    atributos_df.append('clase_e1')

# En caso que se realice el ensamble entre las 3 estrategias
if len(estrategias_feature_extraction)==3:
  TRAIN_FILE_E2 = f'dataset-{estrategias_feature_extraction[2]}-200-prep.csv'
  texto = texto + f'\n\t{TRAIN_FILE_E2}' 
  # Se define el if para que ante reiteradas ejecuciones de la notebook no se vuelva a insertar
  if 'clase_e2' not in atributos_df:
    atributos_df.append('clase_e2')

if len(estrategias_feature_extraction)>=1:
  print(texto)

Los dataset a utilizar son: 
	dataset-lr-200-prep.csv
	dataset-tfidf-200-prep.csv


## 1. Instalación y Carga de librerías y funciones útiles

### 1.1 Instalación de librerías

Se instalan las librerías que no están en el entorno de Google Colab:

In [40]:
!pip install requests
!pip install wget

!pip install simpletransformers

### 1.2 Funciones útiles

Se cargan funciones útiles desde el repo https://github.com/jumafernandez/clasificacion_correos para la carga y balanceo del dataset.

In [41]:
import requests

# Se hace el request del raw del script python
url = 'https://raw.githubusercontent.com/jumafernandez/clasificacion_correos/main/scripts/funciones_dataset.py'
r = requests.get(url)

# Se guarda en el working directory
with open('funciones_dataset.py', 'w') as f:
    f.write(r.text)

# Se importan las funciones a utilizar
from funciones_dataset import get_clases, cargar_dataset, consolidar_df, generar_train_test_set, separar_x_y_rna

También se carga la función para preprocesar el texto que se usó en los otros modelos desde el repo: https://github.com/jumafernandez/clasificacion_correos.

In [42]:
import requests

# Se hace el request del raw del script python
url = 'https://raw.githubusercontent.com/jumafernandez/clasificacion_correos/main/scripts/funciones_preprocesamiento.py'
r = requests.get(url)

# Se guarda en el working directory
with open('funciones_preprocesamiento.py', 'w') as f:
    f.write(r.text)

# Se importan las funciones a utilizar
from funciones_preprocesamiento import preprocesar_correos, preprocesar_correos_bert

### 1.3. Carga de datos

Cargo la librería warnings para no mostrar las advertencias, pandas para el manejo de df y os para verificar la existencia de los archivos en la carga de datos. Además, cargo en memoria la URL de base de los datasets y una lista de las etiquetas de las distintas clases:

In [43]:
import warnings
import pandas as pd
from os import path
warnings.filterwarnings("ignore")

# Constantes con los datos
DS_DIR = 'https://raw.githubusercontent.com/jumafernandez/clasificacion_correos/main/data/50jaiio/consolidados/'

# Defino las clases y la cantidad a utilizar
etiquetas = get_clases()
CANTIDAD_CLASES = len(etiquetas)

Se cargan los dataframe en memoria con el preprocesamiento de los datos:
- Instancias LR,
- Instancias SS3,
- Instancias TFIDF.
- Instancias etiquetadas manualmente.

In [44]:
if len(estrategias_feature_extraction)>=1:
  # Chequeo sobre si los archivos están en el working directory
  download_files = not(path.exists(TRAIN_FILE_E0))

  df_e0, test_df, etiquetas = cargar_dataset(DS_DIR, TRAIN_FILE_E0, TEST_FILE, download_files, 'clase', etiquetas, CANTIDAD_CLASES, 'Otras Consultas')

  # Muestro salida por consola
  print('Existen {} clases: {}.'.format(len(df_e0.clase.unique()), df_e0.clase.unique()))

  # Me quedo con las N cantidad de instancias con mayor score por clase
  if CANTIDAD_INSTANCIAS<200:
    df_e0 = df_e0.sort_values(['clase','score'], ascending=False).groupby('clase').head(CANTIDAD_INSTANCIAS).reset_index(drop=True)
    df_e0 = df_e0.sample(frac = 1)


El conjunto de entrenamiento tiene la dimensión: (3200, 25)
El conjunto de testeo tiene la dimensión: (200, 24)
Existen 16 clases: ['Boleto Universitario' 'Cambio de Carrera' 'Cambio de Comisión'
 'Consulta por Equivalencias' 'Consulta por Legajo'
 'Consulta sobre Título Universitario' 'Cursadas' 'Datos Personales'
 'Exámenes' 'Ingreso a la Universidad' 'Pedido de Certificados'
 'Problemas con la Clave' 'Reincorporación' 'Requisitos de Ingreso'
 'Simultaneidad de Carreras' 'Situación Académica'].


In [45]:
if len(estrategias_feature_extraction)>=2:
  # Chequeo sobre si los archivos están en el working directory
  download_files = not(path.exists(TRAIN_FILE_E1))

  df_e1, test_df_e1, etiquetas = cargar_dataset(DS_DIR, TRAIN_FILE_E1, TEST_FILE, download_files, 'clase', etiquetas, CANTIDAD_CLASES, 'Otras Consultas')

  # Muestro salida por consola
  print('Existen {} clases: {}.'.format(len(df_e1.clase.unique()), df_e1.clase.unique()))

  # Me quedo con las N cantidad de instancias con mayor score por clase
  if CANTIDAD_INSTANCIAS<200:
    df_e1 = df_e1.sort_values(['clase','score'], ascending=False).groupby('clase').head(CANTIDAD_INSTANCIAS).reset_index(drop=True)
    df_e1 = df_e1.sample(frac = 1)


El conjunto de entrenamiento tiene la dimensión: (3200, 25)
El conjunto de testeo tiene la dimensión: (200, 24)
Existen 16 clases: ['Boleto Universitario' 'Cambio de Carrera' 'Cambio de Comisión'
 'Consulta por Equivalencias' 'Consulta por Legajo'
 'Consulta sobre Título Universitario' 'Cursadas' 'Datos Personales'
 'Exámenes' 'Ingreso a la Universidad' 'Pedido de Certificados'
 'Problemas con la Clave' 'Reincorporación' 'Requisitos de Ingreso'
 'Simultaneidad de Carreras' 'Situación Académica'].


Solo en el caso que decida hacer el sistema de votación entre las 3 estrategias inicializo la tercera:

In [46]:
if len(estrategias_feature_extraction)>=3:

  # Chequeo sobre si los archivos están en el working directory
  download_files = not(path.exists(TRAIN_FILE_E2))

  df_e2, test_df_e2, etiquetas = cargar_dataset(DS_DIR, TRAIN_FILE_E2, TEST_FILE, download_files, 'clase', etiquetas, CANTIDAD_CLASES, 'Otras Consultas')

  # Muestro salida por consola
  print('Existen {} clases: {}.'.format(len(df_e2.clase.unique()), df_e2.clase.unique()))

  # Me quedo con las N cantidad de instancias con mayor score por clase
  if CANTIDAD_INSTANCIAS<200:
    df_e2 = df_e2.sort_values(['clase','score'], ascending=False).groupby('clase').head(CANTIDAD_INSTANCIAS).reset_index(drop=True)
    df_e2 = df_e2.sample(frac = 1)

Verifico si voy a acumular los datos etiquetados manualmente a los de etiquetado no supervisado y en caso afirmativo los cargo en memoria:

In [47]:
if ETIQUETADO_MANUAL:
 
  # Chequeo sobre si los archivos están en el working directory
  download_files = not(path.exists(TRAIN_FILE_MANUAL))

  train_df_manual, test_df, etiquetas = cargar_dataset(DS_DIR, TRAIN_FILE_MANUAL, TEST_FILE, download_files, 'clase', etiquetas, CANTIDAD_CLASES, 'Otras Consultas')

  # Muestro salida por consola
  print('Existen {} clases: {}.'.format(len(train_df_manual.clase.unique()), train_df_manual.clase.unique()))


El conjunto de entrenamiento tiene la dimensión: (800, 24)
El conjunto de testeo tiene la dimensión: (200, 24)
Existen 16 clases: ['Boleto Universitario' 'Problemas con la Clave' 'Cursadas'
 'Cambio de Carrera' 'Ingreso a la Universidad' 'Requisitos de Ingreso'
 'Reincorporación' 'Pedido de Certificados' 'Situación Académica'
 'Exámenes' 'Cambio de Comisión' 'Consulta por Legajo'
 'Consulta sobre Título Universitario' 'Datos Personales'
 'Consulta por Equivalencias' 'Simultaneidad de Carreras'].


### 1.4 Sistema de votación entre estrategias de _feature extraction_

En primer lugar, se joinea mediante el texto de la consulta las estrategias encaradas (según sean dos o tres):

In [48]:
import pandas as pd

# Si se utiliza sólo 1 estrategia, se inicializa el df_join con solo esa
if len(estrategias_feature_extraction)==1:
  df_join = df_e0

# Si se utilizan al menos 2 estrategias, se incorpora también la 2da
if len(estrategias_feature_extraction)>=2:
  df_join = pd.merge(df_e0, df_e1, on='consulta', how='left', suffixes=(None, "_x"))

# Si están las 3 estrategias, se incorpora también la 3era
if len(estrategias_feature_extraction)==3:
  df_join = pd.merge(df_join, df_e2, on='consulta', how='left', suffixes=(None, "_y"))

Borro las instancias con faltantes y verifico el resultado la dimensionalidad del df:

In [49]:
if 'df_join' in globals():
  df_join = df_join.dropna()
  
  print(df_join.shape)

  print(df_join.head())

(1876, 49)
                                             consulta  ...               clase_x
6   buenas tardes queria saber porque me dice erro...  ...  Boleto Universitario
7   buen día. quería tramitar el beneficio de bole...  ...  Boleto Universitario
16  hola, hice hace un mes y algo el tramite onlin...  ...  Boleto Universitario
22  ya hice el tramite de la sube, del boleto estu...  ...  Boleto Universitario
23  hola que tal,quisiera saber cuándo cargan el b...  ...  Boleto Universitario

[5 rows x 49 columns]


Renombro las clases en función de las estrategias y me quedo sólo con las instancias en que la clase coincide:

In [50]:
if len(estrategias_feature_extraction)==3:
  df_join.rename(columns={'clase_x': 'clase_e0', 'clase_y': 'clase_e1', 'clase': 'clase_e2'}, inplace=True)
  df_join['match_clase'] = ((df_join['clase_e0'] == df_join['clase_e1']) & (df_join['clase_e1'] == df_join['clase_e2']))
elif len(estrategias_feature_extraction)==2:
  df_join.rename(columns={'clase': 'clase_e0', 'clase_x': 'clase_e1'}, inplace=True)
  df_join['match_clase'] = df_join['clase_e0'] == df_join['clase_e1']
elif len(estrategias_feature_extraction)==1:
  df_join.rename(columns={'clase': 'clase_e0'}, inplace=True)
  # Esto se hace por compatibilidad con las otras alternativas (2 y 3 estrategias)
  df_join['match_clase'] = df_join['clase_e0'] == df_join['clase_e0']

if 'df_join' in globals():
  # Me quedo sólo con las instancias en que la clase coincide
  train_df_join = df_join.query('match_clase == True').reset_index()

  train_df_join.shape

Me quedo solo con los atributos que me interesan (elimino duplicados):

In [51]:
if 'df_join' in globals():
  train_df_join = train_df_join[atributos_df]

  train_df_join.columns

Me quedo sólo con una columna de clase dado que son las 3 iguales:

In [52]:
if 'df_join' in globals():
  # Tomo una clase al azar, dado que coinciden las 3
  train_df_join.rename(columns={'clase_e0': 'clase'}, inplace=True)

  # Elimino las columnas que no necesito
  if len(estrategias_feature_extraction)>=2:
    train_df_join.drop(['clase_e1'], inplace=True, axis=1)

  if len(estrategias_feature_extraction)==3:
    train_df_join.drop(['clase_e2'], inplace=True, axis=1)

  train_df_join.shape

Me guardo la columna del score de Elasticsearch para no generar incompatibilidades:

In [53]:
if 'df_join' in globals():
  score = train_df_join['score']
  train_df_join.drop('score', inplace=True, axis=1)

In [54]:
if len(estrategias_feature_extraction)>=1:
  if ETIQUETADO_MANUAL:
    train_df = pd.concat([train_df_manual, train_df_join], axis=0).reset_index(drop=True)
else:
  train_df = train_df_manual

Muestro el dataframe resultante:

In [55]:
# pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_rows', None)
train_df.head()

,consulta,dia_semana,semana_del_mes,mes,cuatrimestre,anio,hora_discretizada,dni_discretizado,legajo_discretizado,posee_legajo,posee_telefono,carrera_valor,proveedor_correo,cantidad_caracteres,proporcion_mayusculas,proporcion_letras,cantidad_tildes,cantidad_palabras,cantidad_palabras_cortas,proporcion_palabras_distintas,frecuencia_signos_puntuacion,cantidad_oraciones,utiliza_codigo_asignatura,clase
0,quería saber por qué no puedo acceder al bolet...,6,2,3,1,2019,2,7,4,1,1,3,4,107,0.0,0.775701,3,21,12,0.952381,0.000000,1,0,Boleto Universitario
1,no puedo reseterar mi clave de acceso al siste...,4,2,12,2,2015,0,2,1,1,1,5,6,419,0.0,0.809069,0,78,45,0.717949,0.007160,1,0,Problemas con la Clave
2,me dice que ya tengo el beneficio de la sube y...,1,2,4,1,2019,3,8,4,1,1,3,6,96,0.0,0.781250,1,22,15,0.863636,0.000000,1,0,Boleto Universitario
3,"buenos días, quería consultar cuando me puedo ...",3,4,10,2,2018,1,4,2,1,1,3,6,410,0.0,0.819512,5,70,34,0.728571,0.012195,2,0,Cursadas
4,buenas tardes quiero volver a estudiar en la f...,2,1,2,1,2019,0,2,0,0,1,54,6,175,0.0,0.811429,1,32,16,0.875000,0.011429,1,0,Cambio de Carrera


## 1.5 Consolidación del dataset

Se renombran las instancias para que sea compatible con _simpletransformers_:

In [56]:
train_df = train_df[['consulta', 'clase']]
train_df.columns = ['text', 'labels']
test_df = test_df[['consulta', 'clase']]
test_df.columns = ['text', 'labels']

# Muestro salida por consola
print('Existen {} clases: {}.'.format(len(train_df.labels.unique()), train_df.labels.unique()))

Existen 16 clases: ['Boleto Universitario' 'Problemas con la Clave' 'Cursadas'
 'Cambio de Carrera' 'Ingreso a la Universidad' 'Requisitos de Ingreso'
 'Reincorporación' 'Pedido de Certificados' 'Situación Académica'
 'Exámenes' 'Cambio de Comisión' 'Consulta por Legajo'
 'Consulta sobre Título Universitario' 'Datos Personales'
 'Consulta por Equivalencias' 'Simultaneidad de Carreras'].


Codifico la clase en un atributo numérico por compatibilidad con simpletransformers:

In [57]:
from sklearn import preprocessing

le_clase = preprocessing.LabelEncoder()
train_df['labels'] = le_clase.fit_transform(train_df['labels'])

train_df.head()

,text,labels
0,quería saber por qué no puedo acceder al bolet...,0
1,no puedo reseterar mi clave de acceso al siste...,11
2,me dice que ya tengo el beneficio de la sube y...,0
3,"buenos días, quería consultar cuando me puedo ...",6
4,buenas tardes quiero volver a estudiar en la f...,1


Hago lo mismo para las instancias de test:

In [58]:
test_df['labels'] = le_clase.transform(test_df['labels'])

Verificamos la dimensionalidad del dataframe:

In [59]:
train_df.shape

(1998, 2)

# 2. BETO (BERT pre-entrenado por la Universidad de Chile)

Se carga el modelo pre-entrenado de BERT con una definición de hiperparámetros:

In [60]:
from simpletransformers.classification import ClassificationModel

# Cantidad de epochs
epocas = 4

# Hiperparámetros
train_args = {
        'overwrite_output_dir': True,
        'num_train_epochs': epocas,
        'fp16': True,
        'learning_rate': 4e-5,
        'do_lower_case': True,
        'use_early_stopping': True,
        }

# Creamos el ClassificationModel
model = ClassificationModel(
    model_type='bert', 
#    model_name='bert-base-multilingual-cased',
    model_name='dccuchile/bert-base-spanish-wwm-cased',
    num_labels=CANTIDAD_CLASES,
    use_cuda=False,
    args=train_args
)

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchi

### 2.1 Construcción del modelo

En primer lugar se trabaja con un único clasificador que clasifica y testea las instancias en las 16 clases posibles.

Se realiza el entrenamiento del modelo:

In [ ]:
# Modulo para la hora (a efectos de calcular los tiempos de ejecución)
import time

# Se imprime la hora del servidor
hora_servidor = time.strftime('%H:%M:%S', time.localtime())
print(f'Hora de inicio del entrenamiento: {hora_servidor}.')

model.train_model(train_df)


Hora de inicio del entrenamiento: 05:27:44.


  0%|          | 0/1998 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/250 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/250 [00:00<?, ?it/s]

#### 2.2.1 Definición del espacio de búsqueda

Se define el espacio de búsqueda para el ajuste de hiperparámetros del modelo:

In [ ]:
sweep_config = {
    "name": "vanilla-sweep-batch-16",
    "method": "bayes",
    "metric": {"name": "accuracy", "goal": "maximize"},
    "parameters": {
        "num_train_epochs": {"min": 2, "max": 4},
        "learning_rate": {"min": 0, "max": 4e-4},
    },
    "early_terminate": {"type": "hyperband", "min_iter": 6,},
}

Se ejecuta el ajuste de hiperparámetros para cada estrategia de representación en función del espacio de búsqueda:

## 2.2 Testeo del modelo

Se testea el modelo con el método pre-determinado por simpletransformers y además se calculan las métricas a mano para que sea comparable luego con los otros modelos:

In [ ]:
model.eval_model(test_df)

In [ ]:
!mkdir evaluate_dir

In [ ]:
model.evaluate(test_df, "evaluate_dir")

Evaluamos el modelo con las métricas convencionales a partir de la predicción en test:

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, matthews_corrcoef, f1_score, confusion_matrix

# Ejecutamos las predicciones sobre testing
y_pred, raw_outputs = model.predict(list(test_df.text))

Se pasan las etiquetas de las predicciones y las clases de train y test para que los resultados sean más sencillos de analizar:

In [ ]:
train_df.labels = le_clase.inverse_transform(train_df.labels)

test_df.labels = le_clase.inverse_transform(test_df.labels)

y_pred = le_clase.inverse_transform(y_pred)

Se calculan las métricas sobre test set para el paper:

In [ ]:
acc_test = accuracy_score(test_df.labels, y_pred)
precision_test = precision_score(test_df.labels, y_pred, average='weighted')
mcc_test = matthews_corrcoef(test_df.labels, y_pred)
f1_test = f1_score(test_df.labels, y_pred, average='weighted')

# Genero un diccionario con los parámetro y el acc en test
dict_test = {}
dict_test['clasificador'] = 'BETO'
dict_test['accuracy'] = acc_test
dict_test['f1_score'] = f1_test
dict_test['precision'] = precision_test
dict_test['mcc'] = mcc_test
 
# Paso el diccionario a dataframe y lo guardo en un archivo con fecha/hora
results_test = pd.DataFrame([dict_test])
print(f'Estrategia: {estrategias_feature_extraction} - {ETIQUETADO_MANUAL}')
print(results_test)

In [ ]:
import numpy as np
import seaborn as sns; sns.set()
from sklearn import metrics
import matplotlib.pyplot as plt

mat = metrics.confusion_matrix(test_df.labels, y_pred)

sns.heatmap(mat, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=etiquetas, yticklabels=etiquetas)

plt.xlabel('Observada')
plt.ylabel('Predicha');

### 3. Análisis del error

Se consolida un dataframe para hacer el análisis de la performance:

In [ ]:
df_train_error = {'clase': pd.Series(train_df.labels.unique()), 
                  'count_train': train_df.labels.value_counts().reset_index(drop=True)}
  
df_train_error = pd.DataFrame(df_train_error)

# Genero la matriz de confusión
mat = confusion_matrix(test_df.labels, y_pred)
# Calculo el accuracy por clase
avg_class = mat.diagonal()/mat.sum(axis=1)
df_train_error['accuracy'] = pd.Series(avg_class)

# Calculo la cantidad de instancias bien clasificadas
pred_true_class = mat.diagonal()
df_train_error['good_pred'] = pd.Series(pred_true_class)

# Calculo la cantidad de instancias por clase (suma de las rows)
df_train_error['count_test'] = mat.sum(axis=1)

df_train_error

Se realiza un plot para analizar la performance por clase:

In [ ]:
# Cargo las librerías y defino el paño blanco
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="ticks")

# Ploteo el gráfico general
plt.figure(figsize=(8,5))

sns.scatterplot(data=df_train_error, 
                x="count_train", 
                y="accuracy", 
                hue="count_test")

# Pongo los nombres de las clases
for i in range(df_train_error.shape[0]):
#    if df_train_error.count_test[i]> df_train_error.count_test.mean():
      plt.text(x=df_train_error.count_train[i]+0.01,
              y=df_train_error.accuracy[i]+0.01,
              s=df_train_error.clase[i])

plt.title("Análisis gráfico del error por clase")           #title
# plt.xlim(df_error.score.min()-1,df_error.score.max()+1)   #set x limit
plt.ylim(0.6, 1.15)                                            #set y limit
plt.xlabel("Cantidad de instancias de entrenamiento")       #y label
plt.legend(loc='lower right')

## 4. Persistencia del modelo

Se persiste el modelo en Google Drive:

In [ ]:
PERSISTIR = False

if PERSISTIR:
  # Se zipea la salida
  import shutil
  shutil.make_archive('outputs_07_ensamble_beto_210622', 'zip', 'outputs')
  # Se persiste en google colab
  from google.colab import drive
  drive.mount('drive')
  !cp outputs_07_ensamble_beto_210622.zip "drive/My Drive/50jaiio_modelos/"

## Referencias

- Hyperparameter Optimization for Optimum Transformer Models.
 https://towardsdatascience.com/hyperparameter-optimization-for-optimum-transformer-models-b95a32b70949
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
- https://medium.com/analytics-vidhya/ml-pipelines-using-scikit-learn-and-gridsearchcv-fe605a7f9e05